In [2]:
"""
Agent Processing System
Multi-Agent Financial Analysis - Specialized LLM Agents
Processes AAPL stock query with 4 specialized agents
"""

from openai import OpenAI
from typing import Dict, List, Any
from dataclasses import dataclass
import json
from datetime import datetime
import os

In [61]:
import os

# Alpha Vantage (already working)
os.environ["ALPHAVANTAGE_API_KEY"] = "BVGUKZR1MHVS0T6B"

# OpenAI (ADD YOUR KEY HERE)
os.environ["OPENAI_API_KEY"] =  "sk-proj-" # your openAI key here

client = OpenAI(api_key= "sk-proj-") # your openAI key here
print("✓ API keys configured")

✓ API keys configured


In [62]:
@dataclass
class AgentResponse:
    """Standard response format from all agents"""
    agent_name: str                  # Which agent produced this (e.g., "News Analysis Agent")
    analysis: str                    # The actual text analysis/recommendation
    score: float                     # -1(very negative) to +1 scale (very positive)
    confidence: float                # How confident: 0 (not confident) to 1 (very confident)
    key_factors: List[str]           # Bullet points of important findings
    timestamp: str                   # When the analysis was done

In [63]:
class BaseAgent:
    """Base class for all financial agents"""

    def __init__(self, agent_name: str, model: str = "gpt-4"):
        self.agent_name = agent_name # Stores the agent's name
        self.model = model    # Which AI model to use (default: GPT-4)
        self.memory = []   # Empty list to store conversation history

    def call_llm(self, system_prompt: str, user_message: str) -> str:
        """Call LLM with error handling"""
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt}, #Instructions telling the AI how to behave
                    {"role": "user", "content": user_message}  #The actual data/question to analyze
                ],
                temperature=0.3,  # Lower temperature for more consistent analysis
                max_tokens=800     #limits response length
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error in {self.agent_name}: {str(e)}")
            return f"Error processing request: {str(e)}"

    # Add to BaseAgent class
    def add_to_memory(self, interaction):
      """Store conversation history"""
      self.memory.append({
        'timestamp': datetime.now(),
        'input': interaction['input'],
        'output': interaction['output']
        })

    def get_context(self, last_n=5):
      """Retrieve recent context"""
      return self.memory[-last_n:]

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Override in each specialized agent"""
        raise NotImplementedError("Each agent must implement process method")

## NewsAnalysisAgent
This agent reads news headlines about a stock, asks GPT-4 "are these headlines good or bad news?", gets back a sentiment score, and packages it in a standardized format for the orchestrator to use.
For example: if someone whats get sentiment from new, this agent is  reading news and telling if it's bullish or bearish.

In [83]:
class NewsAnalysisAgent(BaseAgent):
    """Analyzes financial news sentiment and impact"""

    def __init__(self):
        super().__init__("News Analysis Agent")
        self.system_prompt = """You are a financial news analyst specializing in sentiment analysis.
Analyze news articles about companies and provide:
1. Overall sentiment score (-1 to +1, where -1 is very negative, 0 is neutral, +1 is very positive)
2. Key factors driving the sentiment
3. Potential impact on stock price

Be objective and consider both positive and negative aspects.
Return response in JSON format with keys: sentiment_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process news data for sentiment analysis"""
        ticker = data.get('ticker', 'AAPL')
        news_articles = data.get('news', [])

        news_summary = "\n".join([
            f"- {article.get('title', '')}: {article.get('summary', '')}"
            for article in news_articles[:5]
        ])

        user_message = f"""Analyze the following recent news about {ticker}:

{news_summary}

Provide sentiment analysis and impact assessment."""

        llm_response = self.call_llm(self.system_prompt, user_message)

        # SAFE PARSING
        try:
            result = json.loads(llm_response)
            score = result.get('sentiment_score', 0)
            analysis = result.get('analysis', llm_response)
            key_factors = result.get('key_factors', [])

            # Safe confidence extraction
            confidence_raw = result.get('confidence', 0.7)
            try:
                confidence = max(0.0, min(1.0, float(confidence_raw)))
            except (ValueError, TypeError):
                confidence = 0.7

        except json.JSONDecodeError:
            score = 0
            analysis = llm_response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.5

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )

## EarningsAnalysisAgent
The Earnings Analysis Agent examines a company's financial performance (revenue, profits, earnings per share) and determines if the fundamentals are strong by comparing actual results against analyst expectations. It sends this financial data to GPT-4, which returns a score (-1 to +1) indicating whether the company's financials suggest it's a good or weak investment.

In [84]:
class EarningsAnalysisAgent(BaseAgent):
    """Analyzes earnings reports and financial statements"""

    def __init__(self):
        super().__init__("Earnings Analysis Agent")
        self.system_prompt = """You are a financial analyst specializing in earnings and fundamental analysis.
Analyze company financial data and provide:
1. Fundamental strength score (-1 to +1, where -1 is very weak, +1 is very strong)
2. Key financial metrics analysis (revenue, earnings, growth)
3. Comparison to expectations
4. Important trends

Return response in JSON format with keys: fundamental_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:  # Fixed indentation
        """Process earnings and financial data"""
        ticker = data.get('ticker', 'UNKNOWN')
        financials = data.get('financials', {})

        # Prepare financial summary
        financial_summary = f"""
Company: {ticker}
Revenue: ${financials.get('revenue', 'N/A')}B
EPS: ${financials.get('eps', 'N/A')}
Revenue Growth: {financials.get('revenue_growth', 'N/A')}%
Profit Margin: {financials.get('profit_margin', 'N/A')}%
Expected Revenue: ${financials.get('expected_revenue', 'N/A')}B
Expected EPS: ${financials.get('expected_eps', 'N/A')}
"""

        user_message = f"""Analyze the following financial data for {ticker}:

{financial_summary}

Assess fundamental strength and growth prospects."""

        llm_response = self.call_llm(self.system_prompt, user_message)

        # SAFE PARSING
        try:
            result = json.loads(llm_response)
            score = result.get('fundamental_score', 0)  # Note: fundamental_score, not sentiment_score
            analysis = result.get('analysis', llm_response)
            key_factors = result.get('key_factors', [])

            # Safe confidence extraction
            confidence_raw = result.get('confidence', 0.8)
            try:
                confidence = max(0.0, min(1.0, float(confidence_raw)))
            except (ValueError, TypeError):
                confidence = 0.8

        except json.JSONDecodeError:
            score = 0
            analysis = llm_response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.6

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )

## MarketSignalsAgent
The MarketSignalsAgent performs technical analysis by examining stock price patterns, trading volume, and technical indicators (like moving averages, RSI, MACD) to identify trends and momentum. It sends this technical data to GPT-4, which returns a score (-1 to +1) indicating whether the stock's price action suggests a bullish or bearish trend based on chart patterns and trading signals.

In [86]:
class MarketSignalsAgent(BaseAgent):
    """Performs technical analysis on market data"""

    def __init__(self):
        super().__init__("Market Signals Agent")
        self.system_prompt = """You are a technical analyst specializing in market signals and price patterns.
Analyze technical indicators and provide:
1. Technical strength score (-1 to +1, where -1 is very bearish, +1 is very bullish)
2. Key technical indicators assessment
3. Support and resistance levels
4. Trend analysis

Return response in JSON format with keys: technical_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process technical market data"""
        ticker = data.get('ticker', 'UNKNOWN')
        technicals = data.get('technicals', {})

        technical_summary = f"""
Ticker: {ticker}
Current Price: ${technicals.get('current_price', 'N/A')}
50-day MA: ${technicals.get('ma_50', 'N/A')}
200-day MA: ${technicals.get('ma_200', 'N/A')}
RSI: {technicals.get('rsi', 'N/A')}
MACD: {technicals.get('macd', 'N/A')}
Volume: {technicals.get('volume', 'N/A')} (Avg: {technicals.get('avg_volume', 'N/A')})
Support: ${technicals.get('support', 'N/A')}
Resistance: ${technicals.get('resistance', 'N/A')}
"""

        user_message = f"""Analyze the following technical data for {ticker}:

{technical_summary}

Assess technical strength and price momentum."""

        llm_response = self.call_llm(self.system_prompt, user_message)

        # SAFE PARSING
        try:
            result = json.loads(llm_response)
            score = result.get('technical_score', 0)
            analysis = result.get('analysis', llm_response)
            key_factors = result.get('key_factors', [])

            # Safe confidence extraction
            confidence_raw = result.get('confidence', 0.7)
            try:
                confidence = max(0.0, min(1.0, float(confidence_raw)))
            except (ValueError, TypeError):
                confidence = 0.7

        except json.JSONDecodeError:
            score = 0
            analysis = llm_response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.5

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )

## RiskAssessmentAgent
The RiskAssessmentAgent evaluates investment risk by analyzing metrics like beta (volatility), Value at Risk, Sharpe ratio, and sector correlation to determine how risky a stock is for a portfolio. It sends these risk metrics to GPT-4, which returns a risk score (0 to 1, where 0 is low risk and 1 is high risk) along with warnings about potential portfolio concentration or volatility issues.

In [87]:
class RiskAssessmentAgent(BaseAgent):
    """Assesses investment risk and portfolio fit"""

    def __init__(self):
        super().__init__("Risk Assessment Agent")
        self.system_prompt = """You are a risk management analyst specializing in portfolio risk assessment.
Analyze risk metrics and provide:
1. Risk level score (0 to 1, where 0 is very low risk, 1 is very high risk)
2. Key risk factors
3. Portfolio diversification implications
4. Risk-adjusted return assessment

Return response in JSON format with keys: risk_score, analysis, key_factors, confidence"""

    def process(self, data: Dict[str, Any]) -> AgentResponse:
        """Process risk metrics"""
        ticker = data.get('ticker', 'UNKNOWN')
        risk_data = data.get('risk_metrics', {})

        risk_summary = f"""
Ticker: {ticker}
Beta: {risk_data.get('beta', 'N/A')}
Volatility (30-day): {risk_data.get('volatility', 'N/A')}%
Value at Risk (5%): ${risk_data.get('var_5', 'N/A')}
Sharpe Ratio: {risk_data.get('sharpe_ratio', 'N/A')}
Max Drawdown: {risk_data.get('max_drawdown', 'N/A')}%
Sector Correlation: {risk_data.get('sector_correlation', 'N/A')}
P/E Ratio: {risk_data.get('pe_ratio', 'N/A')}
"""

        user_message = f"""Analyze the following risk metrics for {ticker}:

{risk_summary}

Assess overall investment risk and portfolio implications."""

        llm_response = self.call_llm(self.system_prompt, user_message)

        # SAFE PARSING
        try:
            result = json.loads(llm_response)
            score = result.get('risk_score', 0.5)
            analysis = result.get('analysis', llm_response)
            key_factors = result.get('key_factors', [])

            # Safe confidence extraction
            confidence_raw = result.get('confidence', 0.8)
            try:
                confidence = max(0.0, min(1.0, float(confidence_raw)))
            except (ValueError, TypeError):
                confidence = 0.8

        except json.JSONDecodeError:
            score = 0.5
            analysis = llm_response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.6

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )

## SynthesisAgent
The SynthesisAgent acts as the "decision maker" that takes all the individual agent analyses (news sentiment, earnings strength, technical signals, risk level) and combines them into a single investment recommendation (STRONG BUY, BUY, HOLD, SELL, STRONG SELL). It sends a summary of all agent scores and findings to GPT-4, which weighs the different perspectives and returns a final actionable recommendation with confidence level and supporting reasoning.

In [88]:
class SynthesisAgent(BaseAgent):
    """Combines insights from all agents into final recommendation"""

    def __init__(self):
        # Initialize with base agent functionality
        super().__init__("Research Synthesis Agent")
        # Define instructions for the AI on how to synthesize multiple analyses
        self.system_prompt = """You are a senior investment analyst who synthesizes multiple analyses into actionable recommendations.
Given analyses from news, earnings, technical, and risk agents, provide:
1. Overall investment recommendation (STRONG BUY, BUY, HOLD, SELL, STRONG SELL)
2. Confidence level (0 to 1)
3. Key reasoning
4. Risk considerations
5. Target price range (if applicable)

Return response in JSON format with keys: recommendation, confidence, analysis, key_points, risks"""

    def process(self, agent_responses: List[AgentResponse]) -> AgentResponse:
        """Synthesize all agent responses"""
        analyses_summary = "\n\n".join([
            f"{resp.agent_name}:\n"
            f"Score: {resp.score}\n"
            f"Analysis: {resp.analysis}\n"
            f"Key Factors: {', '.join(resp.key_factors)}"
            for resp in agent_responses
        ])

        user_message = f"""Synthesize the following analyses into a final investment recommendation:

{analyses_summary}

Provide comprehensive investment recommendation with supporting reasoning."""

        llm_response = self.call_llm(self.system_prompt, user_message)

        # SAFE PARSING
        try:
            result = json.loads(llm_response)
            recommendation = result.get('recommendation', 'HOLD')
            analysis = result.get('analysis', llm_response)
            key_factors = result.get('key_points', [])

            # Safe confidence extraction
            confidence_raw = result.get('confidence', 0.7)
            try:
                confidence = max(0.0, min(1.0, float(confidence_raw)))
            except (ValueError, TypeError):
                confidence = 0.7

            # Convert recommendation to score
            rec_to_score = {
                'STRONG BUY': 1.0,
                'BUY': 0.6,
                'HOLD': 0.0,
                'SELL': -0.6,
                'STRONG SELL': -1.0
            }
            score = rec_to_score.get(recommendation, 0.0)

        except json.JSONDecodeError:
            score = 0
            analysis = llm_response
            key_factors = ["Unable to parse structured response"]
            confidence = 0.6

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(score),
            confidence=float(confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )

## CritiqueAgent

The CritiqueAgent acts as a "quality control checker" that reviews the final investment recommendation to catch mistakes, biases, or missing information before presenting it to the user. It examines the SynthesisAgent's recommendation, asks GPT-4 to identify logical flaws or gaps in reasoning, and can adjust the confidence level downward if it finds issues (like  "didn't consider macroeconomic factors"), ensuring the final output is reliable and well-reasoned.


In [89]:
class CritiqueAgent(BaseAgent):
    """Reviews and validates analysis quality"""

    def __init__(self):
        super().__init__("Critique & Validation Agent")
        self.system_prompt = """You are critique analyst who reviews investment recommendations for biases, logical errors, and completeness.
Review the synthesis and identify:
1. Logical inconsistencies
2. Potential biases
3. Missing considerations
4. Data quality issues
5. Confidence adjustment recommendation

Return response in JSON format with keys: quality_score, issues_found, suggestions, adjusted_confidence"""

    def process(self, synthesis_response: AgentResponse) -> AgentResponse:
        """Critique the synthesis"""
        user_message = f"""Review this investment analysis for quality and completeness:

Recommendation: {synthesis_response.analysis}
Confidence: {synthesis_response.confidence}
Key Factors: {', '.join(synthesis_response.key_factors)}

Identify any issues, biases, or missing elements."""

        llm_response = self.call_llm(self.system_prompt, user_message)

        # SAFE PARSING
        try:
            result = json.loads(llm_response)
            quality_score = result.get('quality_score', 0.7)
            issues = result.get('issues_found', [])
            suggestions = result.get('suggestions', [])
            adjusted_confidence_raw = result.get('adjusted_confidence', synthesis_response.confidence)

            # Safe confidence extraction
            try:
                adjusted_confidence = max(0.0, min(1.0, float(adjusted_confidence_raw)))
            except (ValueError, TypeError):
                adjusted_confidence = synthesis_response.confidence

            analysis = f"Quality Score: {quality_score}\n"
            if issues:
                analysis += f"Issues Found: {', '.join(issues)}\n"
            if suggestions:
                analysis += f"Suggestions: {', '.join(suggestions)}"

            key_factors = issues if issues else ["No major issues found"]

        except json.JSONDecodeError:
            quality_score = 0.7
            analysis = llm_response
            adjusted_confidence = synthesis_response.confidence
            key_factors = ["Unable to parse structured response"]

        return AgentResponse(
            agent_name=self.agent_name,
            analysis=analysis,
            score=float(quality_score),
            confidence=float(adjusted_confidence),
            key_factors=key_factors,
            timestamp=datetime.now().isoformat()
        )

In [90]:
# -*- coding: utf-8 -*-
"""notebooks/01_data_ingestion.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1J_syHGmJHhA8XcAtxw7nQ0KPWMJbnSkd
"""

#import os
os.environ["ALPHAVANTAGE_API_KEY"] = "BVGUKZR1MHVS0T6B"

import os
import requests
import pandas as pd
import yfinance as yf
from datasets import Dataset

# ------------------------------
# Helper: Convert Pandas → Hugging Face Dataset
# ------------------------------
def to_hf(df, schema=None):
    """Convert a pandas DataFrame to a Hugging Face Dataset. Handles empty gracefully."""
    if df is None or getattr(df, "empty", True):
        if schema:
            return Dataset.from_dict({c: [] for c in schema})
        return Dataset.from_dict({})
    if schema:
        df = df[[c for c in schema if c in df.columns]].copy()
    return Dataset.from_pandas(df.reset_index(drop=True), preserve_index=False)

# ------------------------------
# Alpha Vantage Connector (for news + indicators only)
# ------------------------------
class AlphaConnector:
    def __init__(self, api_key=None):
        # Pick up API key from os.environ if not passed directly
        self.api_key = api_key or os.getenv("ALPHAVANTAGE_API_KEY")
        if not self.api_key:
            raise ValueError("Alpha Vantage API key not found. Set os.environ['ALPHAVANTAGE_API_KEY'].")

        self.base_url = "https://www.alphavantage.co/query"

    def fetch_news(self, symbol):
        """Fetch company news & sentiment (Alpha Vantage)."""
        params = {
            "function": "NEWS_SENTIMENT",
            "tickers": symbol,
            "apikey": self.api_key
        }
        r = requests.get(self.base_url, params=params)
        data = r.json()

        if "feed" not in data:
            print("No news data:", data)
            return pd.DataFrame()

        rows = []
        for item in data["feed"]:
            rows.append({
                "published_at": item.get("time_published"),
                "source": item.get("source"),
                "title": item.get("title"),
                "summary": item.get("summary"),
                "url": item.get("url"),
                "overall_sentiment": item.get("overall_sentiment_label"),
                # ** Added By Ali **
                "overall_sentiment_score": item.get("overall_sentiment_score") # both label and score so later agents (NewsAnalysisAgent, SynthesisAgent, etc.) can use either
            })
        return pd.DataFrame(rows)

    def fetch_indicator(self, symbol, indicator, interval="daily", time_period=14, series_type="close"):
        """Generic technical indicator fetch (SMA, RSI, MACD)."""
        params = {
            "function": indicator,
            "symbol": symbol,
            "interval": interval,
            "time_period": time_period,
            "series_type": series_type,
            "apikey": self.api_key
        }
        r = requests.get(self.base_url, params=params)
        data = r.json()

        key_map = {
            "SMA": "Technical Analysis: SMA",
            "RSI": "Technical Analysis: RSI",
            "MACD": "Technical Analysis: MACD"
        }
        key = key_map.get(indicator)
        if key not in data:
            print(f"{indicator} fetch failed:", data)
            return pd.DataFrame()

        df = pd.DataFrame.from_dict(data[key], orient="index")
        df.index = pd.to_datetime(df.index)
        df.reset_index(inplace=True)
        df = df.rename(columns={"index": "date"})

        # Cast numeric values
        for col in df.columns:
            if col != "date":
                df[col] = df[col].astype(float)

        return df

# ------------------------------
# Data Ingestion Manager
# ------------------------------
class DataIngestionManager:
    def __init__(self, api_key=None):
        self.alpha = AlphaConnector(api_key)

    def fetch_all(self, symbol, start=None, end=None):
        """Fetch prices (Yahoo), news (Alpha Vantage), SMA, RSI (Alpha Vantage)."""
        datasets = {}

        # Prices from Yahoo Finance (unlimited)
        try:
            df_prices = yf.download(symbol, start=start, end=end, progress=False)

            # Flatten MultiIndex columns if necessary
            if isinstance(df_prices.columns, pd.MultiIndex):
                df_prices.columns = [c[0].lower() for c in df_prices.columns]

            df_prices = df_prices.reset_index().rename(columns={
                "Date": "date",
                "open": "open",
                "high": "high",
                "low": "low",
                "close": "close",
                "adj close": "adj_close",
                "volume": "volume"
            })
            df_prices["date"] = df_prices["date"].astype(str)

            datasets["prices"] = to_hf(
                df_prices, schema=["date","open","high","low","close","adj_close","volume"]
            )
        except Exception as e:
            print("Yahoo Finance fetch failed:", e)
            datasets["prices"] = to_hf(pd.DataFrame(), schema=["date","open","high","low","close","adj_close","volume"])

        # News from Alpha Vantage
        datasets["news"] = to_hf(
            self.alpha.fetch_news(symbol),
            schema=["published_at","source","title","summary","url","overall_sentiment"]
        )

        # Technical Indicators from Alpha Vantage
        datasets["sma"] = to_hf(
            self.alpha.fetch_indicator(symbol, "SMA", time_period=20),
            schema=["date","SMA"]
        )
        datasets["rsi"] = to_hf(
            self.alpha.fetch_indicator(symbol, "RSI", time_period=14),
            schema=["date","RSI"]
        )

        # Removed MACD to avoid premium-only error
        return datasets

from datetime import datetime, timedelta

mgr = DataIngestionManager()  # will pick up the key from os.environ
symbol = "AAPL"
start = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
end   = datetime.now().strftime("%Y-%m-%d")

datasets = mgr.fetch_all(symbol, start, end)

print("Prices sample:")
print(datasets["prices"].to_pandas().head())

print("News sample:")
print(datasets["news"].to_pandas().head())

/tmp/ipython-input-4032339001.py:120: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_prices = yf.download(symbol, start=start, end=end, progress=False)


No news data: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
SMA fetch failed: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
RSI fetch failed: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Prices sample:
         date        open        high         low       close     volume
0  2025-09-15  237.000000  238.190002  235.029999  236.699997   42699500
1  2025-09-16  237.179993  241

In [91]:
# ============================================================================
# INTEGRATION TEST CELL
# ============================================================================

def test_engineer1_engineer3_integration():
    """Test Engineer 1's real data with Engineer 3's real agents"""

    print("="*80)
    print("INTEGRATION TEST - Engineer 1 + Engineer 3")
    print("="*80)

    # Get data from Engineer 1
    print("\n[STEP 1] Fetching real data...")
    from datetime import datetime, timedelta

    mgr = DataIngestionManager()
    symbol = "AAPL"
    start = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
    end = datetime.now().strftime("%Y-%m-%d")

    datasets = mgr.fetch_all(symbol, start, end)
    print(f"✓ Fetched data for {symbol}")

    # Convert to agent format
    print("\n[STEP 2] Preparing data for agents...")
    news_df = datasets["news"].to_pandas()
    prices_df = datasets["prices"].to_pandas()
    sma_df = datasets["sma"].to_pandas()
    rsi_df = datasets["rsi"].to_pandas()

    # News input
    news_articles = news_df.head(5).to_dict('records')
    news_input = {'ticker': symbol, 'news': news_articles}

    # Technical input
    latest_price = prices_df.iloc[-1] if len(prices_df) > 0 else {}
    latest_sma = sma_df.iloc[-1] if len(sma_df) > 0 else {}
    latest_rsi = rsi_df.iloc[-1] if len(rsi_df) > 0 else {}

    technicals_input = {
        'ticker': symbol,
        'technicals': {
            'current_price': str(latest_price.get('close', 'N/A')),
            'ma_50': str(latest_sma.get('SMA', 'N/A')),
            'rsi': str(latest_rsi.get('RSI', 'N/A')),
            'volume': str(latest_price.get('volume', 'N/A'))
        }
    }

    # Financial input (mock for now)
    financials_input = {
        'ticker': symbol,
        'financials': {
            'revenue': '394.3',
            'eps': '6.42',
            'revenue_growth': '15.0',
            'profit_margin': '26.3'
        }
    }

    # Risk input (mock for now)
    risk_input = {
        'ticker': symbol,
        'risk_metrics': {
            'beta': '1.15',
            'volatility': '22.5',
            'sharpe_ratio': '1.35'
        }
    }

    print("✓ Data prepared")

    # Test agents
    print("\n[STEP 3] Testing agents with REAL OpenAI...")

    news_agent = NewsAnalysisAgent()
    news_resp = news_agent.process(news_input)
    print(f"✓ News: {news_resp.score:.2f}")

    earnings_agent = EarningsAnalysisAgent()
    earn_resp = earnings_agent.process(financials_input)
    print(f"✓ Earnings: {earn_resp.score:.2f}")

    market_agent = MarketSignalsAgent()
    market_resp = market_agent.process(technicals_input)
    print(f"✓ Market: {market_resp.score:.2f}")

    risk_agent = RiskAssessmentAgent()
    risk_resp = risk_agent.process(risk_input)
    print(f"✓ Risk: {risk_resp.score:.2f}")

    # Synthesize
    print("\n[STEP 4] Synthesizing recommendation...")
    synthesis_agent = SynthesisAgent()
    all_responses = [news_resp, earn_resp, market_resp, risk_resp]
    final_resp = synthesis_agent.process(all_responses)
    print(f"✓ Final Score: {final_resp.score:.2f}")

    # Critique
    print("\n[STEP 5] Critiquing analysis...")
    critique_agent = CritiqueAgent()
    critique_resp = critique_agent.process(final_resp)
    print(f"✓ Quality: {critique_resp.score:.2f}")

    # Results
    print("\n" + "="*80)
    print("FINAL RESULTS")
    print("="*80)
    print(f"\nTicker: {symbol}")
    print(f"\nAgent Scores:")
    for resp in all_responses:
        print(f"  {resp.agent_name:30s} {resp.score:+.2f}")
    print(f"\nFinal: {final_resp.score:+.2f} (Confidence: {final_resp.confidence:.0%})")
    print(f"Quality: {critique_resp.score:.2f}")
    print("\n✓ TEST COMPLETE!")

# Run it!
test_engineer1_engineer3_integration()

INTEGRATION TEST - Engineer 1 + Engineer 3

[STEP 1] Fetching real data...


/tmp/ipython-input-4032339001.py:120: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_prices = yf.download(symbol, start=start, end=end, progress=False)


No news data: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
SMA fetch failed: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
RSI fetch failed: {'Information': 'We have detected your API key as BVGUKZR1MHVS0T6B and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
✓ Fetched data for AAPL

[STEP 2] Preparing data for agents...
✓ Data prepared

[STEP 3] Testing agents with REAL OpenAI...
✓ News: 0.00
✓ Earnings: 0.85
✓ Market: 0.00
✓ Risk: 0.57

[STEP 4]